In [ ]:

import csv
import urllib
import numpy as np
import pandas as pd
import psycopg2
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import email
from email import encoders
from email.mime.base import MIMEBase
import json
import urllib.parse

def Read_Contexts():
    myvars = {}
    with open("/DE/Decisioning_DE/context_DT") as myfile:
        for line in myfile:
            name, var = line.partition(";~")[::2]
            if len(var)>0:
                myvars[name.strip()] = var.rstrip(var[-1])
            else:
                myvars[name.strip()] = var
    return myvars


def Get_Dates_and_WSA_Data(conn, param_st_dt,param_end_dt):
    print(1)
# =============================================================================
#     stmt = """
#     SELECT COALESCE(MAX(log_time),'2022-10-01'::date) as max_date
#     FROM decisioning_de.processed_responses_v2
#     """
# =============================================================================
    if param_st_dt =='' or param_end_dt=='':

# =============================================================================
#         cursor = conn.cursor()
#         cursor.execute(stmt)
#         StartDate = cursor.fetchone()[0]
#         cursor.close()
#         StartDate=StartDate.date()
# =============================================================================
        StartDate= datetime.now().date() 
        TodayDate = datetime.now().date() + timedelta(days=1)
    else:
        StartDate = param_st_dt
        TodayDate = param_end_dt
    
 #   yesterday_date = (TodayDate - timedelta(days=1))
 #  if StartDate != yesterday_date:
 #       print("Data is Updated")
 #   else:
    print("StartDate: " + str(StartDate) +"\n" + "EndDate: " + str(TodayDate))
    stmt_read_webService_api = f"""
    select log_time, log_message->'details'->>'client' client,log_message->'details'->>'req_json' client_request, 
    log_message->'details'->>'response' response 
    from decisioning_app.web_service_api wsa 
    where wsa.log_message->'details'->>'path'='/public/get_decision' 
    AND log_time >= '{str(StartDate)}' and log_time < '{str(TodayDate)}'
    """
    df=pd.read_sql_query(stmt_read_webService_api, conn)
    return df,StartDate,TodayDate
    
def Dodgy_Responses(df_wsa,engine,  conn,  StartDate, TodayDate ):
    df_dodgy_stg= df_wsa.query("response=='' | response=='{}'")
    df_dodgy_stg['log_time'] = df_dodgy_stg['log_time'].apply(pd.to_datetime)
    df_dodgy = df_dodgy_stg.groupby(df_dodgy_stg['log_time'].dt.date).size().reset_index(name='Count')
    df_dodgy.rename(columns={'log_time': "log_date", "Count": "dodgy_count"}, inplace=True)
    cursor=conn.cursor()
    try:
        cursor.execute(f"delete FROM decisioning_de.dodgy_responses WHERE log_date >= '{str(StartDate)}' and log_date < '{str(TodayDate)}'")
        cursor.close()
        conn.commit()
        print("inserting data")
        df_dodgy.to_sql('dodgy_responses', con = engine, schema='decisioning_de', if_exists = 'append', chunksize = 1000, index = False)
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
def parse_request_response(df_wsa,df_stg):
    for index,row in df_wsa.iterrows():
        log_time = df_wsa['log_time'][index]
        request_string = df_wsa['client_request'].iloc[index]
        json_request = json.loads(request_string)
        response_string = df_wsa['response'].iloc[index]
        json_response = json.loads(response_string)
        intent=""
        offergroups_req=""
        offer_type=""
        mban=""
        afiniti_rank=0
        original_rank=0
        rank_change=0
        sub_offer_string=""
        offergroups_res=""
        shuffle =0
        n_propositions=0

        if response_string!='{}' and response_string!='': #and json_request.get('offerRecord') is not None and json_response.get('offerRecord') is not None:
            for offerRecord_req,offerRecord_res in zip(json_request['offerRecord'], json_response['offerRecord']): 
                n_propositions=0
                tdata_id = json_request['transactionId']
                mban = json_response['mBan']
                sub_offer_list=[]
                sub_offer_string=""
                if offerRecord_req.get('dispositionLevel') is not None and offerRecord_req.get('inventorySpaceId') is not None:

                    if offerRecord_req['dispositionLevel']=='Group' and offerRecord_req['inventorySpaceId'].find('Reactive')!=-1:
                        if int(offerRecord_res['Rank'])-int(offerRecord_req['Rank'])!=0:
                            shuffle =1
                        else:
                            shuffle =0
                        for subOffer in offerRecord_req['offerGroupMembers']:
                            sub_offer_list.append(subOffer['offerGroupMember'])
                        sub_offer_list.sort()
                        sub_offer_string = '|'.join(sub_offer_list)

                        row = [log_time, mban, tdata_id,offerRecord_req['intent'],offerRecord_req['offerGroup'],offerRecord_req['inventorySpaceId'],sub_offer_string,int(offerRecord_req['Rank']) ,int(offerRecord_res['Rank']),len(sub_offer_list),int(offerRecord_res['Rank'])-int(offerRecord_req['Rank']),shuffle]
                        df_length = len(df_stg)
                        df_stg.loc[df_length] = row
        else:
            continue
    return df_stg    


In [ ]:
###################################################### READ CONTEXT ###################################################################
myvars = Read_Contexts()
EMAIL_TO = myvars['Email_To_US'].split(',' or ';')
EMAIL_SUBJECT = 'ATT - Response/Request Code'
EMAIL_FROM = myvars['Email_From']
EMAIL_SMTP = myvars['Email_Host']
EMAIL_PORT = myvars['Email_Port']
special_password =  urllib.parse.quote(myvars['GP_Password'])
print('contexts Loaded')

###################################################### CREATE CONNECTION ################################################################
conn = psycopg2.connect(host=myvars['GP_Server'], dbname=myvars['GP_DB'], user=myvars['GP_User'], password=myvars['GP_Password'], port=myvars['GP_Port'])
conn2 = psycopg2.connect(host=myvars['GP_Server'], dbname=myvars['GP_DB'], user=myvars['GP_User'], password=myvars['GP_Password'], port=myvars['GP_Port'])
conn_string = 'postgresql+psycopg2://'+myvars['GP_User']+':'+special_password+'@'+myvars['GP_Server']+':'+str(myvars['GP_Port'])+'/'+myvars['GP_DB']
engine = create_engine(conn_string)
print('Connections Created')

####################################################### GET WSA DATA #####################################################################
#param_st_dt = datetime.strptime('2022-10', '%d-%m-%Y').date()
#param_st_dt = dt.date(2022, 10, 15)
#param_ed_dt = dt.date(2022, 10, 16)

param_st_dt =''
param_ed_dt =''


df_wsa,StartDate,TodayDate = Get_Dates_and_WSA_Data(conn, param_st_dt, param_ed_dt)
conn.commit()
print('DF_WSA LOADED')

In [ ]:
####################################################### PARSE REQUEST/RESPONSE ###########################################################

df_stg=pd.DataFrame(columns=['log_time','mban','tdata_id', 'intent','offer_group','offer_type','offer_group_members','original_rank','afiniti_rank', 'n_propositions','rank_change','shuffle'])
df_stg = parse_request_response(df_wsa,df_stg)
print('Response/Request Parsed')
workingdir = "/dbstorage/offload_de/"
csvfile = workingdir+"processed_responses.csv"
f=open(csvfile,'wb') # opens file for writing (erases contents)
df_stg.to_csv(csvfile, sep='~', encoding='utf-8', index=False)
f.close()
cursor = conn2.cursor()
cursor.execute(f"delete FROM decisioning_de.processed_responses_v2 WHERE log_time >= '{str(StartDate)}' and log_time < '{str(TodayDate)}'")
cursor.close()
conn.commit() 
cursor = conn2.cursor()
cursor.execute("""INSERT INTO decisioning_de.processed_responses_v2 SELECT log_time::timestamp(3), mban, tdata_id, intent, offer_group, offer_type, offer_group_members, original_rank::int,
afiniti_rank::int, n_propositions::int, rank_change::int, shuffle::int FROM decisioning_de.ext_processed_responses;""")
cursor.close()
conn2.commit()


######################################################## DODGY RESPONSES ##################################################################

Dodgy_Responses(df_wsa, engine,conn, StartDate, TodayDate)
print('Dodgy responses Done')
''''

######################################################### Final Table Insertion ###########################################################

    try:
        print("inserting data")
        df_stg.to_sql('processed_responses_v2', con = engine, schema='decisioning_de', if_exists = 'append', chunksize = 1000, index = False)
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    conn.close()
    conn2.close()
else:
    print("Data is Already Up to Date")
'''''